In [57]:
### vdb 업데이트를 위한 크롤링된 csv 파일 chk
import pandas as pd
import os
import sys
import time
import requests
import json
import re
import random


In [58]:
table = pd.read_csv("crawling_results.csv")

In [101]:
table.head()

,url,job_category,job_name,title,company_name_raw,company_name,location,experience_requirement,company_id,description,main_work,qualification,preferences,welfare,deadline,tag_name
0,https://www.wanted.co.kr/wd/323905,10110,소프트웨어 엔지니어,백엔드 엔지니어,클래스101(Class101)∙서울 강남구∙경력 3년 이상,클래스101(Class101),서울 강남구,경력 3년 이상,/company/3257,"세상에 없던 길을 만들어온 팀, CLASS101입니다. 2018년, 준비물까지 챙겨...",주요업무 합류시 함께 하게 될 Platform & Infrastructure 팀은 ...,자격요건 •개발 경력 3년 이상 •AWS 등 클라우드 인프라 활용 경험 •데이터베이...,"우대사항 •Full-Stack 개발 경험 (React, TypeScript, Nod...","혜택 및 복지 #근무환경 * 완전자율근무제: 단, 팀/업무 특성에 따라 리드의 판단...",2026-01-15,"51~300명, 설립4~9년, 식대지원, 커피·스낵바, 대출지원, 자기계발지원, 육..."
1,https://www.wanted.co.kr/wd/265172,10110,소프트웨어 엔지니어,빅데이터 솔루션 데이터 엔지니어 (5년 이상),로이드케이∙경기 성남시∙경력 5-16년,로이드케이,경기 성남시,경력 5-16년,/company/36974,로이드케이팀은 기술로 상상을 현실로 만드는 사람들입니다. 데이터·AI·클라우드·보안...,주요업무 • 대용량/실시간 데이터 수집/처리 및 모니터링 시스템 설계/운영 • 키워...,자격요건 • Python/Shell Script 기반 데이터 파이프라인 자동화 경험...,"우대사항 • 컴퓨터 공학, 시스템 공학 등 관련 전공 • 기술 컨설팅, 고객사 대응...",혜택 및 복지 • 내가 원하는 커피/간식을 마음대로 구매해요. (복지카드/월 5만원...,2026-01-15,"유망산업, 커피·스낵바, 50명이하, 인원 급성장, 설립10년이상, 연봉 업계평균이..."
2,https://www.wanted.co.kr/wd/318508,10110,소프트웨어 엔지니어,Fullstack SW Engineer,딥메트릭스∙서울 강남구∙경력 3-8년,딥메트릭스,서울 강남구,경력 3-8년,/company/21714,"DeepMetrics는 2020년, 서울대학교 컴퓨터공학부 송현오 교수와 서울대학교...",주요업무 • 의료기기 소프트웨어(SaMD) 서비스 개발 및 운영 • 의료 관련 기술...,자격요건 • 소프트웨어 개발(풀스택) 경력 3년 이상 • 3년 이상의 RDBMS 사...,우대사항 • CI/CD 환경에서의 개발 및 배포 경험 • IEC62304 등 의료기...,혜택 및 복지 • 유연 근무제 운영 • 최신 IT 장비 및 소프트웨어 지원 • 점심...,2026-01-15,"50명이하, 연봉상위6~10%, 설립3년이하, 보너스, 스톡옵션, 식대지원"
3,https://www.wanted.co.kr/wd/323516,10110,소프트웨어 엔지니어,AI Optimization Engineer (최적화 엔지니어),네오사피엔스(타입캐스트)∙서울 강남구∙경력 3-6년,네오사피엔스(타입캐스트),서울 강남구,경력 3-6년,/company/4666,"[ 세상에서 ​가장 ​쉬운 ​콘텐츠 제작 ​인공지능(AI), 타입캐스트 ] 네오사피...",주요업무 • Inference & Training Computational Opti...,자격요건 • PyTorch / TensorRT / ONNX 개발 경험 • C++ 코...,우대사항 • PyTorch / TensorRT / ONNX 에 대한 깊은 이해 • ...,NaN,2026-01-15,"유망산업, 커피·스낵바, 자기계발지원, 51~300명, 설립4~9년, 인원 급성장,..."
4,https://www.wanted.co.kr/wd/321873,10110,소프트웨어 엔지니어,Forward Deployed ML Engineer,리얼월드∙서울 강남구∙경력 3년 이상,리얼월드,서울 강남구,경력 3년 이상,/company/51934,리얼월드(RLWRLD)는 로봇이 인간처럼 세상을 인식하고 사고하며 손을 써서 작업할...,주요업무 - 산업 환경에 적합한 Fine-tuning 및 Domain adaptat...,"자격요건 - AI/Robotics 분야에서 3년 이상, ML을 실제 Edge/Rob...","우대사항 - Linux 환경 및 Python, C++ 기반 ML/Inference ...",혜택 및 복지 유연근무제 업무용 소프트웨어 구입 지원 무제한 간식 제공 경조사/명절...,2026-01-15,"커피·스낵바, 유망산업, AI 선도 기업, 누적투자100억이상"


In [60]:
for col in table.columns:
    print(col)
print(f"총 {len(table.columns)}개의 열이 있습니다.")


url
job_category
job_name
title
company_name_raw
company_name
location
experience_requirement
company_id
description
main_work
qualification
preferences
welfare
deadline
tag_name
총 16개의 열이 있습니다.


In [61]:
table["deadline"]

0            상시채용
1            상시채용
2            상시채용
3            상시채용
4            상시채용
          ...    
100    2026.01.31
101          상시채용
102          상시채용
103          상시채용
104          상시채용
Name: deadline, Length: 105, dtype: object

In [62]:
from collections import Counter
deadline_counts = Counter(table["deadline"])
print(deadline_counts)
for d in deadline_counts:
    print(d, deadline_counts[d])


Counter({'상시채용': 93, '2025.12.31': 6, '2026.02.17': 2, '2025.12.21': 1, '2025.12.14': 1, '2025.12.27': 1, '2026.01.31': 1})
상시채용 93
2026.02.17 2
2025.12.21 1
2025.12.31 6
2025.12.14 1
2025.12.27 1
2026.01.31 1


In [63]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

def parse_deadline(x):
    x = str(x).strip()
    
    # 1) 날짜 형식이면: YYYY.MM.DD
    try:
        dt = datetime.strptime(x, "%Y.%m.%d")
        return dt.replace(hour=0, minute=0, second=0, microsecond=0)
    except:
        pass

    # 2) 날짜가 아니면: 현재 기준 + 1개월
    return (datetime.now() + relativedelta(months=1)).replace(hour=0, minute=0, second=0, microsecond=0)

# 예시 데이터프레임
table["deadline"] = table["deadline"].apply(parse_deadline)


In [64]:
table["deadline"].head()

0   2026-01-15
1   2026-01-15
2   2026-01-15
3   2026-01-15
4   2026-01-15
Name: deadline, dtype: datetime64[ns]

In [65]:
now = datetime.now()
deadline = table["deadline"]
is_expired = deadline < now

In [66]:
is_expired.head()

0    False
1    False
2    False
3    False
4    False
Name: deadline, dtype: bool

In [67]:
# ### 12월 7일 기준 jd-dev 컬럼
#   "ID": "0d31eccd-c434-44e8-915a-a7844b9919bd",
#   "company": "더즌(dozn)",
#   "experience_requirement": "경력 3-7년",
#   "job_url": "https://www.wanted.co.kr/wd/310004",
#   "location": "서울 서초구",
#   "summary": "### 주요 직무 및 목표 ... ",
#   "text":전문:


### 현재 chunk metadata 구성부분, text와 summary는 별도로 추가되니 고려 x
# for i, desciption in enumerate(sentences):
#     meta_data = [{"job_url": table.iloc[i]["url"],
#     "company": table.iloc[i]["company_name"],
#     "location": table.iloc[i]["location"],
#     "experience_requirement": table.iloc[i]["experience_requirement"],
#     "summary": table.iloc[i]["summary"]
#     }]

### Goal
- 만료기간도 pinecone에서 사용하도록 추가
- 이 기간이 없는 경우에 대한 처리 추가(일단 1개월 잡고 나중에 고도화 하는걸로)
- 만료기간 적당한거 넣어서 삭제되는지 chk`
- crawling.csv에서 파인콘 upsert 까지 한번의 step으로 완료

### 정합성 chk
- csv 저장하는 코드 다듬어서 현재일 기준 csv로 저장되도록
- URL 모두 가져오기 테스트&중복 체크


In [ ]:
from dotenv import load_dotenv
import pandas as pd
import os
from pinecone import Pinecone, ServerlessSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from tqdm import tqdm

pc = Pinecone(api_key="PINECONE_API_KEY")


In [69]:
index_name = "korea-jd-dev"
index = pc.Index(index_name)

In [107]:
all_ids = []
for batch in index.list():
    for vid in batch:
        all_ids.append(vid)

print(len(all_ids))
print(all_ids[:10])

1282
['00743b99-ab3f-4e86-a30d-a290e02e9412', '00b1b763-9c6c-4278-a803-737ea6ae12f8', '00dc745b-d9ec-41d5-8575-a2e7f6ac4481', '010e677a-92ea-4429-9a70-dd6badcc82c3', '01406f59-8015-443f-a542-16a907fce073', '0161915e-5223-4dc1-8945-ad766dc82ea3', '019ddce1-b120-4564-8d83-9ac2b62561aa', '01bcc269-0eb5-4840-b45d-2c1911de3c99', '02179ea7-dc95-4d31-ac28-7cfbd1386440', '02398c2f-5e9a-467b-bbea-89753fc8b6e7']


In [108]:
def get_all_ids(index):
    all_ids = []
    for batch in index.list():
        for vid in batch:
            all_ids.append(vid)
    return all_ids

In [109]:
def get_metadata_by_id(index, vid):
    resp = index.fetch(ids=[vid])
    vectors = resp.vectors[vid]
    metadata = vectors.metadata
    return metadata  # 없으면 None

In [110]:
url_set = set()
date_dicts = {}

for ids in tqdm(all_ids):
    # print(get_metadata_by_id(index, ids))
    row = get_metadata_by_id(index, ids)
    url_set.add(row["job_url"])
    date_dicts[ids] = row["deadline"]

100%|██████████| 1282/1282 [04:33<00:00,  4.68it/s]


In [111]:
from datetime import datetime

In [112]:
def check_deadline(deadline_str):
    # 날짜 형식에 맞게 변환
    try:
        deadline = datetime.strptime(deadline_str, "%Y-%m-%d")
    except ValueError:
        return False  # 파싱 불가(상시채용 등) 시 False 반환
    ### 현재시간보다 미래일시 True, 과거일시 False, 최종적으로 False는 모두 제거해야함
    return deadline > datetime.now()

In [113]:
delete_ids = []
for k, v in date_dicts.items():
    if check_deadline(v)==False:
        delete_ids.append(k)

In [114]:
delete_ids

[]

In [89]:
def delete_vectors(index, ids):
    """
    Pinecone에서 벡터 삭제
    
    Args:
        index: Pinecone index 객체
        ids: 단일 ID(str) 또는 ID 리스트(list)
    
    Returns:
        삭제된 ID 개수
    """
    # 단일 ID면 리스트로 변환
    if isinstance(ids, str):
        ids = [ids]
    
    # 배치 삭제 (한 번에 최대 1000개)
    batch_size = 1000
    deleted_count = 0
    
    for i in range(0, len(ids), batch_size):
        batch = ids[i:i+batch_size]
        index.delete(ids=batch)
        deleted_count += len(batch)
    
    print(f"✅ {deleted_count}개의 벡터가 삭제되었습니다.")
    return deleted_count


### 사용 예시

```python
# 1. 단일 ID 삭제
delete_vectors(index, "0021d604-1980-4b4e-b570-30727e77d8a1")

# 2. 여러 ID 삭제
ids_to_delete = ["id1", "id2", "id3"]
delete_vectors(index, ids_to_delete)

# 3. 만료된 항목 삭제 (date_dicts에서 필터링)
expired_ids = [vid for vid, deadline in date_dicts.items() if not check_deadline(deadline)]
delete_vectors(index, expired_ids)
```


In [91]:
delete_vectors(index, delete_ids)

✅ 29개의 벡터가 삭제되었습니다.


29

### 전처리(summary) 추가 확인

In [92]:
url_set

{'https://www.wanted.co.kr/wd/134575',
 'https://www.wanted.co.kr/wd/141545',
 'https://www.wanted.co.kr/wd/142217',
 'https://www.wanted.co.kr/wd/158312',
 'https://www.wanted.co.kr/wd/193161',
 'https://www.wanted.co.kr/wd/218542',
 'https://www.wanted.co.kr/wd/221103',
 'https://www.wanted.co.kr/wd/239334',
 'https://www.wanted.co.kr/wd/246217',
 'https://www.wanted.co.kr/wd/247032',
 'https://www.wanted.co.kr/wd/251783',
 'https://www.wanted.co.kr/wd/253325',
 'https://www.wanted.co.kr/wd/256242',
 'https://www.wanted.co.kr/wd/258728',
 'https://www.wanted.co.kr/wd/264052',
 'https://www.wanted.co.kr/wd/268260',
 'https://www.wanted.co.kr/wd/273281',
 'https://www.wanted.co.kr/wd/273728',
 'https://www.wanted.co.kr/wd/275708',
 'https://www.wanted.co.kr/wd/278395',
 'https://www.wanted.co.kr/wd/278650',
 'https://www.wanted.co.kr/wd/279162',
 'https://www.wanted.co.kr/wd/281342',
 'https://www.wanted.co.kr/wd/281358',
 'https://www.wanted.co.kr/wd/282657',
 'https://www.wanted.co.k

In [93]:
preprocessed_df = pd.read_csv("/home/yhkim/code/JobPT/preprocess/crawling_results_preprocessed_1215.csv")

In [94]:
col = preprocessed_df.columns
for c in col:
    print(c)
print(f"총 {len(col)}개의 열이 있습니다.")

url
job_category
job_name
title
company_name_raw
company_name
location
experience_requirement
company_id
description
main_work
qualification
preferences
welfare
deadline
tag_name
summary
총 17개의 열이 있습니다.


In [95]:
preprocessed_df["url"]

0      https://www.wanted.co.kr/wd/323905
1      https://www.wanted.co.kr/wd/265172
2      https://www.wanted.co.kr/wd/318508
3      https://www.wanted.co.kr/wd/323516
4      https://www.wanted.co.kr/wd/321873
                      ...                
100    https://www.wanted.co.kr/wd/299138
101    https://www.wanted.co.kr/wd/322797
102    https://www.wanted.co.kr/wd/139191
103    https://www.wanted.co.kr/wd/318364
104    https://www.wanted.co.kr/wd/315982
Name: url, Length: 105, dtype: object

In [96]:
new_df = preprocessed_df[~preprocessed_df["url"].isin(url_set)]

print(f"전체 데이터: {len(preprocessed_df)}개")
print(f"Pinecone에 이미 있는 URL: {len(url_set)}개")
print(f"새로운 데이터: {len(new_df)}개")
print(f"제거된 데이터: {len(preprocessed_df) - len(new_df)}개")

전체 데이터: 105개
Pinecone에 이미 있는 URL: 190개
새로운 데이터: 45개
제거된 데이터: 60개


In [97]:
new_df.to_csv("/home/yhkim/code/JobPT/preprocess/crawling_results_preprocessed_1215_new.csv", index=False)

In [98]:
temp = preprocessed_df[preprocessed_df["url"].isin(url_set)]


In [100]:
temp["url"]

20     https://www.wanted.co.kr/wd/324623
23     https://www.wanted.co.kr/wd/321959
29     https://www.wanted.co.kr/wd/322469
31     https://www.wanted.co.kr/wd/322877
33     https://www.wanted.co.kr/wd/315669
35     https://www.wanted.co.kr/wd/323834
36     https://www.wanted.co.kr/wd/312858
37     https://www.wanted.co.kr/wd/315669
39     https://www.wanted.co.kr/wd/324128
40     https://www.wanted.co.kr/wd/318805
41     https://www.wanted.co.kr/wd/319604
42     https://www.wanted.co.kr/wd/319027
43     https://www.wanted.co.kr/wd/239334
44     https://www.wanted.co.kr/wd/322625
45     https://www.wanted.co.kr/wd/324599
46     https://www.wanted.co.kr/wd/283351
48     https://www.wanted.co.kr/wd/319521
49     https://www.wanted.co.kr/wd/322529
50     https://www.wanted.co.kr/wd/283337
51     https://www.wanted.co.kr/wd/323834
52     https://www.wanted.co.kr/wd/325390
53     https://www.wanted.co.kr/wd/239334
54     https://www.wanted.co.kr/wd/322469
56     https://www.wanted.co.kr/wd